# Modeling – Fraud Detection

## Task 2: Model Building, Training, and Evaluation

**Objective:**  
Train and evaluate classification models for fraud detection using metrics appropriate for highly imbalanced data.

# Load Processed Data

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    average_precision_score
)


ModuleNotFoundError: No module named 'pandas'